In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
cd /content/drive/MyDrive/app/

/content/drive/MyDrive/app


In [2]:
import cv2,os
data_path='/content/drive/MyDrive/app/cnn_dataset/training_set/'
categories=os.listdir(data_path)
labels=[i for i in range(len(categories))]

label_dict=dict(zip(categories,labels))
print(label_dict)
print(categories)
print(labels)

{'dogs': 0, 'cats': 1}
['dogs', 'cats']
[0, 1]


In [ ]:
img_size=100
data=[]
target=[]

for category in categories:
    folder_path=os.path.join(data_path,category)
    img_names=os.listdir(folder_path)
        
    for img_name in img_names:
        img_path=os.path.join(folder_path,img_name)
        img=cv2.imread(img_path)

        try:  
            resized=cv2.resize(img,(img_size,img_size))
            #resizing the image  into 100x100, since we need a fixed common size for all the images in the dataset
            data.append(resized)
            target.append(label_dict[category])
            #appending the image and the label(categorized) into the list (dataset)
        except Exception as e:
            print('Exception:',e)
            #if any exception rasied, the exception will be printed here. And pass to the next image

In [ ]:
import numpy as np
data=np.array(data)/255.0
data=np.reshape(data,(data.shape[0],img_size,img_size,3))
target=np.array(target)
from keras.utils import np_utils
new_target=np_utils.to_categorical(target)

In [ ]:
new_target.shape

(400, 2)

In [ ]:
data.shape

(400, 100, 100, 3)

In [ ]:
data.shape[1:]

(100, 100, 3)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten,Dropout
from keras.layers import Conv2D,MaxPooling2D
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam

model=Sequential()

model.add(Conv2D(200,(3,3),input_shape=data.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#The first CNN layer followed by Relu and MaxPooling layers

model.add(Conv2D(100,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#The second convolution layer followed by Relu and MaxPooling layers

model.add(Flatten())
model.add(Dropout(0.5))
#Flatten layer to stack the output convolutions from second convolution layer
model.add(Dense(50,activation='relu'))
#Dense layer of 64 neurons
model.add(Dense(2,activation='softmax'))
#The Final layer with two outputs for two categories

model.compile(loss='categorical_crossentropy', optimizer='Adam' ,metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 98, 98, 200)       5600      
_________________________________________________________________
activation_6 (Activation)    (None, 98, 98, 200)       0         
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 49, 49, 200)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 47, 47, 100)       180100    
_________________________________________________________________
activation_7 (Activation)    (None, 47, 47, 100)       0         
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 23, 23, 100)       0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 52900)            

In [ ]:
from sklearn.model_selection import train_test_split
train_data,test_data,train_target,test_target=train_test_split(data,new_target,test_size=0.1)

In [ ]:
train_data.shape

(360, 100, 100, 3)

In [ ]:
train_target.shape

(360, 2)

In [ ]:
history=model.fit(train_data,train_target,epochs=20,validation_split=0.2)

Epoch 1/20
9/9 [==============================] - 22s 2s/step - loss: 0.8665 - accuracy: 0.5036 - val_loss: 0.6912 - val_accuracy: 0.4722
Epoch 2/20
9/9 [==============================] - 21s 2s/step - loss: 0.6883 - accuracy: 0.5581 - val_loss: 0.6949 - val_accuracy: 0.4722
Epoch 3/20
9/9 [==============================] - 21s 2s/step - loss: 0.6855 - accuracy: 0.5309 - val_loss: 0.7628 - val_accuracy: 0.4722
Epoch 4/20
9/9 [==============================] - 21s 2s/step - loss: 0.6900 - accuracy: 0.5630 - val_loss: 0.6810 - val_accuracy: 0.6250
Epoch 5/20
9/9 [==============================] - 21s 2s/step - loss: 0.6323 - accuracy: 0.6534 - val_loss: 0.7226 - val_accuracy: 0.5139
Epoch 6/20
9/9 [==============================] - 21s 2s/step - loss: 0.5772 - accuracy: 0.6735 - val_loss: 0.6868 - val_accuracy: 0.6111
Epoch 7/20
9/9 [==============================] - 21s 2s/step - loss: 0.4797 - accuracy: 0.7967 - val_loss: 0.7500 - val_accuracy: 0.6111
Epoch 8/20
9/9 [==================

In [ ]:
model.save('/content/drive/MyDrive/app/cnn_model.h5')

In [5]:
from tensorflow import keras
model = keras.models.load_model('/content/drive/MyDrive/app/cnn_model.h5')

In [6]:
from  keras.preprocessing  import image

In [ ]:
test_img = image.load_img('/content/drive/MyDrive/app/cnn_dataset/single_prediction/cat_or_dog3.jpg')

In [ ]:
test_img

In [ ]:
test_img_arr = image.img_to_array(test_img)

In [ ]:
import numpy

In [ ]:
test_image_arr_4d   = numpy.expand_dims(test_img_arr, axis=0)

In [ ]:
test_image_arr_4d.shape

In [ ]:
model.predict(test_image_arr_4d)